In [1]:
from copy import deepcopy
from math import log
import queue  # 用来层序打印决策树
import time  # 用来记录运行时间

In [2]:
class TreeNode():
    def __init__(self, attributeIndex, attributeValue=None, isContinue=False, threshold=None):
        self.attributeIndex: int = attributeIndex
        self.attributeValue: str = attributeValue  # 离散属性值
        self.childs = []
        self.isLeaf = False
        self.label = ''
        self.isContinue = isContinue
        self.threshold: int = threshold  # 连续属性划分值

In [3]:
def CleanData():
    with open('./data1/adult.data', 'r') as f:
        with open('./data1/adult_clean.data', 'w') as output:
            for line in f:
                if line == '\n':
                    continue
                if '?' in line:
                    continue
                # words = Line(line)
                words = line.split(',')
                for word in words[:-2]:
                    output.write(word)
                output.write(words[-1])

In [4]:
CleanData()

In [5]:
# 读取训练集
data = open('./data1/adult_clean.data', 'r')
lines = data.readlines()

In [6]:
# 读取数据，去掉换行
Labels = []
sampleIndex = []  # 0-30161
for i in range(len(lines)):
    sampleIndex.append(i)
    lines[i] = lines[i].strip('\n')
    if (lines[i].split(' ')[-1] == '<=50K'):
        Labels.append(0)
    else:
        Labels.append(1)

# 最后一行会有空行
lines = lines[:-1]
sampleIndex = sampleIndex[:-1]

allAttributesIndex = []  # 0-13
for i in range(len(lines[0].split(' '))):
    allAttributesIndex.append(i)
allAttributesIndex = allAttributesIndex[:-1]
continueAttributeIndex = [0, 4, 10, 11, 12]  # 若2作为连续值，计算量太大，需要加速计算，在生成树函数中体现, 不使用性能较好，使用的话可以在数组里添加2
discreteAttributeIndex = [1, 3, 5, 6, 7, 8, 9]  # 12 为离散值的时候可能会出现没有出现过的值，所以作为连续值比较好

In [7]:
def MostLabel(sample: list):
    """
    选出sample中最多的类别
    :param sample: sample 索引
    :return: 类别
    """
    positive = 0
    negtive = 0
    for i in sample:
        if (Labels[i] == '<=50K'):
            negtive += 1
        else:
            positive += 1
    if positive > negtive:
        return '1'
    else:
        return '0'

In [8]:
def CreateDicisionTree(samplesIndex: list, discreteAttributeIndex: list, continueAttributeIndex: list,
                       root: TreeNode = None, deep: int = 0, reUse: bool = True):
    """
    生成一棵 DicisionTree
    :param samplesIndex: 结点的数据索引
    :param discreteAttributeIndex: 离散的属性索引
    :param continueAttributeIndex: 连续属性索引
    :param root: 当前结点
    :return: None 创建时传入一个头节点，生成以该结点为根节点的DecisionTree
    """
    # 减枝
    # if (len(samplesIndex) < 10):   # 这种方法效果不好
    #     root.isLeaf = True
    #     root.label = MostLabel(samplesIndex)
    #     return
    if (reUse and deep > 20):
        root.isLeaf = True
        root.label = MostLabel(samplesIndex)
        return
    if (discreteAttributeIndex == []):
        root.isLeaf = True
        root.label = MostLabel(samplesIndex)
        return
    isSame = True
    l = Labels[samplesIndex[0]]
    for i in samplesIndex:
        if Labels[i] != l:
            isSame = False
            break
    if isSame:
        root.label = l
        root.isLeaf = True
        return
    # 一个存储 Sample 的 dict
    # 第一个索引为attribute 的编号 int
    # 第二个索引为attribute 所有的取值，连续的多一个记录 ['min_ent']
    # 第三个索引连续值和离散值不同
    # 离散{'positive':0,'negtive':0，'sample':[]}
    # 连续{'sample':sample_index,'label':int(flag)}
    dict_attribute_index_attribute_class = []
    # 计算所有属性的Ent，以及每个属性对应的sample index
    for i in range(len(allAttributesIndex)):
        dict_attribute_index_attribute_class.append({})

    for i in samplesIndex:
        flag = 0
        if (Labels[i]):
            flag = 1
        attributes = lines[i].split(' ')
        # print(attributes)
        # 计算离散属性的Ent
        # 离散 map 数据结构
        # index:{'name':{'positive':0,'negtive':0，'sample':[]}，... , }
        for j in discreteAttributeIndex:
            if (dict_attribute_index_attribute_class[j].get(attributes[j])):
                if (flag):
                    dict_attribute_index_attribute_class[j][attributes[j]]['positive'] += 1
                    dict_attribute_index_attribute_class[j][attributes[j]]['sample'].append(i)
                else:
                    dict_attribute_index_attribute_class[j][attributes[j]]['negtive'] += 1
                    dict_attribute_index_attribute_class[j][attributes[j]]['sample'].append(i)
            else:
                if (flag):
                    dict_attribute_index_attribute_class[j][attributes[j]] = {'positive': 1, 'negtive': 0,
                                                                              'sample': [i]}
                else:
                    dict_attribute_index_attribute_class[j][attributes[j]] = {'positive': 0, 'negtive': 1,
                                                                              'sample': [i]}
        # 连续值处理
        # 连续值 map 数据结构
        # index:{ value:{'sample':sample_index,'label':int(flag)} }
        for j in continueAttributeIndex:
            # 需要用到的东西，每个属性的总样本量，以及里面的正负样本
            if (dict_attribute_index_attribute_class[j].get(int(attributes[j]))):
                if flag:
                    dict_attribute_index_attribute_class[j][int(attributes[j])]['positive'] += 1
                    dict_attribute_index_attribute_class[j][int(attributes[j])]['sample'][i] = flag
                else:
                    dict_attribute_index_attribute_class[j][int(attributes[j])]['negtive'] += 1
                    dict_attribute_index_attribute_class[j][int(attributes[j])]['sample'][i] = flag
            else:
                if flag:
                    dict_attribute_index_attribute_class[j][int(attributes[j])] = {'sample': {i: flag}, 'positive': 1,
                                                                                   'negtive': 0}
                else:
                    dict_attribute_index_attribute_class[j][int(attributes[j])] = {'sample': {i: flag}, 'positive': 0,
                                                                                   'negtive': 1}

    # 连续计算错误
    for j in continueAttributeIndex:
        dict_attribute_index_attribute_class[j]['min_ent'] = (999, 99999)  # Ent curValue
        key_sorted = sorted(list(dict_attribute_index_attribute_class[j].keys())[:-1])
        # 选出Ent最小的划分
        for i in range(0, len(key_sorted) - 1):
            # 取出当前的划分
            # 这里取得是Value
            cur_value = float((key_sorted[i] + key_sorted[i + 1]) / 2)
            front_negtive = 0
            front_positve = 0
            l = 0
            while (l < len(key_sorted) and key_sorted[l] < cur_value):
                # 找到Value 对应的sample index   # 找到的sample 太少了
                # 计算属性值小于这个的Ent
                front_negtive += dict_attribute_index_attribute_class[j][key_sorted[l]]['negtive']
                front_positve += dict_attribute_index_attribute_class[j][key_sorted[l]]['positive']
                # l+=1
                # 加速第三个属性计算
                if (j == 2):
                    l += 10
                else:
                    l += 1

            front_num = front_positve + front_negtive
            # 要记录当前的值和index
            if (front_positve == 0):
                front = 0
            else:
                front = -(front_positve / (front_positve + front_negtive) * log(
                    front_positve / (front_positve + front_negtive)))
            if (front_negtive == 0):
                later = 0
            else:
                later = - (front_negtive / (front_positve + front_negtive) * log(
                    front_negtive / (front_positve + front_negtive)))
            front_ent = front + later

            later_negtive = 0
            later_positve = 0
            l = -1
            while (l >= -len(key_sorted) and key_sorted[l] >= cur_value):
                later_negtive += dict_attribute_index_attribute_class[j][key_sorted[l]]['negtive']
                later_positve += dict_attribute_index_attribute_class[j][key_sorted[l]]['positive']
                # l-=1
                # 加速第三个属性计算
                if (j == 2):
                    l -= 10
                else:
                    l -= 1
            later_num = later_positve + later_negtive
            # 要记录当前的值和index
            if (later_positve == 0):
                front = 0
            else:
                front = -(later_positve / (later_positve + later_negtive) * log(
                    later_positve / (later_positve + later_negtive)))
            if (later_negtive == 0):
                later = 0
            else:
                later = - (later_negtive / (later_positve + later_negtive) * log(
                    later_negtive / (later_positve + later_negtive)))
            later_ent = front + later
            # 计算加权Ent，要选出最小的Ent
            ENT = (front_num / (front_num + later_num)) * front_ent + (later_num / (front_num + later_num)) * later_ent
            if ENT < dict_attribute_index_attribute_class[j]['min_ent'][0]:
                dict_attribute_index_attribute_class[j]['min_ent'] = (ENT, cur_value)
            # 加速第三个属性计算
            if (j == 2):
                i += 100
            # Ent ContinueAttributeIndex cur_value

    # 计算连续值
    is_continue = False
    min_index = 999999
    min_ent = 999999
    discreteAttri = []
    for i in allAttributesIndex:
        if i in discreteAttributeIndex:
            if (dict_attribute_index_attribute_class[i] == {}):
                continue
            else:
                ent, weight_ent = Ent(dict_attribute_index_attribute_class[i])
                discreteAttri.append(weight_ent)
                if weight_ent < min_ent:
                    min_ent = weight_ent
                    min_index = i
                    is_continue = False
        if i in continueAttributeIndex:
            discreteAttri.append(999999)
            if dict_attribute_index_attribute_class[i]['min_ent'][0] < min_ent:
                min_ent = dict_attribute_index_attribute_class[i]['min_ent'][0]
                min_index = i
                is_continue = True

    # 选出weight_ent 最小的一个属性当作结点
    # 要从当前的属性索引列表中去掉这个属性索引，再进行子节点构建
    # 要判断是否为连续的属性
    node_attri_index = min_index

    # print('Node', node_attri_index)
    front_index = []
    later_index = []

    if is_continue:
        # 如果是连续的话，还需要知道前后的索引
        # 计算前后索引
        key_sorted = sorted(list(dict_attribute_index_attribute_class[node_attri_index].keys())[:-1])
        l = 0
        while (l < len(key_sorted) and key_sorted[l] <
               dict_attribute_index_attribute_class[node_attri_index]['min_ent'][1]):
            front_index += dict_attribute_index_attribute_class[node_attri_index][key_sorted[l]]['sample']
            l += 1
        l = -1
        while (l >= -len(key_sorted) and key_sorted[l] >=
               dict_attribute_index_attribute_class[node_attri_index]['min_ent'][1]):
            later_index += dict_attribute_index_attribute_class[node_attri_index][key_sorted[l]]['sample']
            l -= 1

    # 不是连续值
    if not is_continue:
        for key in dict_attribute_index_attribute_class[node_attri_index]:
            root.childs.append(TreeNode(attributeIndex=node_attri_index, attributeValue=key))
        for i in root.childs:
            temp = deepcopy(discreteAttributeIndex)
            # 离散值只分类一次，要去掉离散索引
            temp.remove(node_attri_index)
            CreateDicisionTree(dict_attribute_index_attribute_class[i.attributeIndex][i.attributeValue]['sample'], temp,
                               continueAttributeIndex, i, deep=deep + 1, reUse=reUse)
    else:
        if (front_index != [] and later_index != []):
            # 是否重复使用连续属性，如果使用需要将
            if reUse:
                temp = continueAttributeIndex
            else:
                temp = deepcopy(continueAttributeIndex)
                temp.remove(node_attri_index)
            # 使用二分法
            for i in range(2):
                root.childs.append(
                    TreeNode(attributeIndex=node_attri_index, isContinue=True,
                             threshold=dict_attribute_index_attribute_class[node_attri_index]['min_ent'][1]))
            for i in range(2):
                # 取得先后的索引
                # 离散和连续的属性索引不变
                if i == 0:  # 小于的时候
                    CreateDicisionTree(front_index, discreteAttributeIndex, temp, root.childs[i],
                                       deep=deep + 1, reUse=reUse)
                else:
                    CreateDicisionTree(later_index, discreteAttributeIndex, temp, root.childs[i],
                                       deep=deep + 1, reUse=reUse)
        else:
            # 如果都为空，说明属性相同，直接设置即可
            root.isLeaf = True
            root.label = MostLabel(samplesIndex)

In [9]:
def Ent(attribute: dict):
    all = 0
    classNum = []
    subEnt = []
    # 各个属性的Ent
    for key in attribute:
        positive = attribute[key]['positive']
        negtive = attribute[key]['negtive']
        if (positive == 0):
            front = 0
        else:
            front = -(positive / (positive + negtive) * log(positive / (positive + negtive)))
        if (negtive == 0):
            later = 0
        else:
            later = - (negtive / (positive + negtive) * log(negtive / (positive + negtive)))
        ent = front + later
        subEnt.append(ent)
        classNum.append(positive + negtive)
    # 统计总量
    for i in classNum:
        all += i
    # 加权Ent
    weightEnt = 0
    entIndex = 0
    for key in attribute:
        belong_num = attribute[key]['positive'] + attribute[key]['negtive']
        weightEnt += (belong_num / all) * subEnt[entIndex]
        entIndex += 1
    return subEnt, weightEnt

In [10]:
# 测试
def Test(line):
    cur_node = deepcopy(root)
    words = line.split(',')
    for i in range(len(words)):
        words[i] = words[i].strip(' ')
    print(words)
    while (cur_node.isLeaf != True):
        # 在这里需要判断是否是连续的属性变量
        attribute_id = cur_node.childs[0].attributeIndex
        cur_attribute = words[attribute_id]
        if attribute_id in discreteAttributeIndex:
            find = False
            for i in cur_node.childs:
                if i.attributeValue == cur_attribute:
                    cur_node = i
                    find = True
                    break
            # 无法分类的时候
            if (not find):
                return '0', False;
        else:
            # 变为数字
            cur_attribute = int(cur_attribute)
            nodeAttribute = cur_node.childs[0].threshold
            if cur_attribute <= nodeAttribute:
                cur_node = cur_node.childs[0]
            else:
                cur_node = cur_node.childs[1]
    pred = False
    if (words[-1] == '>50K' and cur_node.label == 1) or (words[-1] == '<=50K' and cur_node.label == 0):
        pred = True
    return cur_node.label, pred

In [11]:
def printTree(root):
    q = queue.Queue()
    q.put(root)
    while (not q.empty()):
        size = q.qsize()
        while (size):
            size -= 1
            cur = q.get()
            if (cur == '|'):
                print(cur, end='')
                continue
            print(cur.attributeIndex, end=' ')
            for i in cur.childs:
                q.put(i)
            q.put('|')
        print('\n')

In [13]:
if __name__ == '__main__':
    # 训练集读取在程序开头
    # 构建DecisionTree
    time_start = time.time()
    root = TreeNode(attributeIndex=-1, attributeValue='root')
    CreateDicisionTree(sampleIndex, discreteAttributeIndex, continueAttributeIndex, root)
    time_end_1 = time.time()
    time_sum = time_end_1 - time_start
    print('构造树的时间', time_sum)
    # 打印树结点信息
    printTree(root)
    # 测试数据集
    data = open('./data1/adult.test', 'r')
    # 测试集第一行会有一行不是数据，从文件中删掉后不需要这句
    data.readline()
    right = 0
    all = 0
    no = 0

    time_start_all = time.time()
    for line in data:
        time_start = time.time()
        if '?' in line:
            continue
        # 数据集最后有空行
        if len(line) < 20:
            continue
        line = line.strip('\n')
        line = line.strip('.')
        res, pred = Test(line)
        if pred:
            right += 1
        all += 1
        time_end_1 = time.time() 
        time_sum = time_end_1 - time_start  
        print('time:', time_sum, 's')
        print('acc:', (right / all) * 100, '%')
        print('进度:', no * 100 / 15060, '%')
        no += 1

    time_end_1 = time.time()  
    time_sum = time_end_1 - time_start_all 
    print('FINISH! \n', (right / all) * 100, '%')

构造树的时间 2.1184146404266357
-1 

7 7 7 7 7 7 |

10 10 |3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 |6 6 6 6 6 6 6 6 6 6 6 6 6 |10 10 |10 10 |6 6 6 6 6 6 6 6 6 6 6 6 6 6 |

3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 |1 1 1 1 1 1 |10 10 |10 10 |10 10 |10 10 |6 6 6 6 6 6 6 6 6 6 6 6 |10 10 |12 12 |10 10 |10 10 |10 10 |6 6 6 6 6 6 6 6 6 6 6 6 6 |10 10 |6 6 6 6 6 6 6 6 6 6 6 6 |6 6 6 6 6 6 6 6 6 |6 6 6 6 6 6 6 6 6 6 6 6 6 ||10 10 |3 3 3 3 3 3 3 3 3 3 3 |10 10 |3 3 3 3 3 3 3 3 3 3 3 3 |10 10 |3 3 3 3 3 3 3 3 3 |10 10 |10 10 |3 3 3 3 3 3 3 ||3 3 3 3 3 3 ||3 3 3 3 3 |5 5 5 5 5 5 5 |0 0 |3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 |6 6 6 6 6 6 6 6 6 6 |10 10 |8 8 8 8 8 |5 5 5 |1 1 1 1 1 ||1 1 1 1 |0 0 ||5 5 5 5 5 5 |1 1 |||3 3 3 3 3 3 3 3 3 3 ||

6 6 6 6 6 6 6 6 6 6 6 6 6 6 |6 6 6 6 6 6 6 6 6 6 6 6 |6 6 6 6 6 6 6 6 6 6 6 6 6 |6 6 6 6 6 6 6 6 6 6 6 6 6 |0 0 |0 0 |6 6 6 6 6 |6 6 6 6 6 6 6 6 6 6 6 |11 11 ||0 0 |6 6 6 6 6 6 6 6 6 6 6 |12 12 ||12 12 |||12 12 |||||11 11 |0 0 |12 12 |12 12 |6 6 6 6 6 6 6 6 6 6 6 6 |0 0 |6 6 6 6 6 6 6 6 6

||5 5 5 5 5 |0 0 |0 0 |1 1 1 1 |5 5 5 5 5 ||0 0 |1 1 1 |||12 12 |||0 0 |0 0 ||0 0 ||0 0 ||12 12 |||12 12 |0 0 ||||5 5 5 5 5 |5 5 ||0 0 |||0 0 |||12 12 ||||12 12 ||12 12 |0 0 |||0 0 |0 0 ||||0 0 |0 0 ||||11 11 ||0 0 ||5 5 5 5 5 |11 11 |12 12 ||||12 12 |12 12 ||0 0 |12 12 |||0 0 |||1 1 1 1 1 1 |6 6 6 6 |0 0 |1 1 1 1 1 1 |9 9 |12 12 |0 0 |0 0 |10 10 |0 0 |8 8 8 8 |12 12 ||5 5 5 5 |5 5 5 5 |0 0 |||1 1 1 1 1 ||5 5 5 5 ||0 0 |12 12 |0 0 |0 0 |||0 0 |||12 12 ||0 0 ||0 0 |0 0 |||||12 12 |11 11 |||12 12 ||5 5 5 5 5 |0 0 |0 0 |0 0 |1 1 1 1 |0 0 ||5 5 5 5 5 ||0 0 ||||0 0 |0 0 |||12 12 ||||||||0 0 |||||0 0 |8 8 8 ||0 0 ||||12 12 |||12 12 ||0 0 ||||0 0 |0 0 ||5 5 ||||0 0 ||1 1 1 1 1 1 ||8 8 ||0 0 ||9 9 |5 5 ||||0 0 |||0 0 |0 0 |||9 9 |||10 10 ||||||||0 0 ||||0 0 |1 1 |||||||||||||1 1 1 1 1 |10 10 |1 1 1 1 |12 12 ||12 12 |12 12 |0 0 |8 8 8 ||||12 12 ||||||0 0 |0 0 |12 12 |12 12 |0 0 |12 12 ||0 0 |||0 0 |0 0 ||||0 0 |12 12 |||8 8 8 |||||0 0 ||||||0 0 ||||0 0 ||||0 0 |11 11 ||0 0 ||0 0 ||0 0 |0 0 ||0 

|||0 0 |||||0 0 |0 0 |12 12 ||0 0 ||1 1 1 |0 0 |||0 0 |0 0 |||12 12 ||||8 8 ||11 11 |0 0 ||||12 12 |||0 0 |||5 5 |||0 0 ||5 5 ||||||8 ||||||||||||0 0 |12 12 |5 5 5 ||0 0 |||12 12 ||||||0 0 |0 0 |0 0 ||||0 0 ||0 0 ||||0 0 ||10 10 ||12 12 |12 12 ||0 0 |||||0 0 |||0 0 |||9 9 |0 0 |11 11 |0 0 |0 0 ||0 0 |||0 0 |11 11 |||0 0 ||||||||0 0 ||||||1 1 ||||||1 |||9 9 ||0 0 |||0 0 ||5 5 5 5 ||||0 0 |||0 0 ||||||0 0 |||||0 0 ||12 12 |||0 0 ||12 12 |12 12 |0 0 |0 0 ||0 0 ||0 0 |||||9 9 |0 0 |||0 0 |0 0 |9 9 |||||5 5 5 ||0 0 |0 0 ||0 0 |||0 0 ||||12 12 ||||||0 0 |||0 0 |0 0 ||0 0 ||||||||0 0 ||0 0 |0 0 |11 11 ||10 10 |0 0 |0 0 |0 0 |8 8 8 |||8 8 |0 0 |||12 12 |||||12 12 |8 8 8 8 8 |0 0 |0 0 |0 0 |0 0 |0 0 ||8 8 |||||||0 0 |||0 0 |8 8 ||0 0 |0 0 |||0 0 ||12 12 |10 10 |12 12 |||0 0 |8 8 ||0 0 ||0 0 ||1 1 1 1 ||8 8 |0 0 ||0 0 |0 0 ||0 0 |||8 8 |0 0 |0 0 |0 0 |0 0 ||1 1 1 1 1 1 |||0 0 |0 0 ||||0 0 |||0 0 |||||0 0 |||||0 0 |||0 0 |5 |0 0 |0 0 |10 10 ||0 0 |0 0 |12 12 ||12 12 |||||0 0 |12 12 ||0 0 |||0 0 |

||8 |9 ||0 0 |11 11 ||9 ||||0 0 ||9 ||||||12 12 |8 |5 5 5 ||||||8 8 |0 0 ||||9 9 |9 9 ||||5 5 |12 12 ||0 0 |5 5 5 5 |||||0 0 |0 0 ||12 12 ||0 0 |||8 8 8 ||12 12 ||||||0 0 |||||0 0 ||0 0 ||||0 0 |||||||9 9 ||10 10 |8 ||11 11 ||0 0 ||0 0 ||9 9 |0 0 ||||||||||0 0 ||12 12 |||8 |8 8 |||8 8 ||12 12 ||0 0 |10 10 |12 12 |0 0 |0 0 |0 0 |||8 8 |12 12 |12 12 |||||0 0 ||5 ||0 0 |0 0 ||0 0 |||0 0 ||12 12 |9 ||0 0 ||0 0 |0 0 |||12 12 ||0 0 ||0 0 |5 |||0 0 |||0 0 |||0 0 ||0 0 |0 0 |5 ||12 12 |12 12 |0 0 ||0 0 ||0 0 |12 12 ||9 |12 12 |0 0 |12 12 |||0 0 |0 0 |12 12 |12 12 ||0 0 |||||0 0 |0 0 ||||0 0 |12 12 |||0 0 |0 0 |||0 0 ||||0 0 |||8 |12 12 |0 0 |12 12 ||0 0 |0 0 |0 0 |0 0 ||0 0 |11 11 |||0 0 ||0 0 |||10 10 |0 0 ||12 12 ||9 |||0 0 ||10 10 |11 11 ||12 12 |||5 |0 0 ||5 |||0 0 |12 12 |12 12 |11 11 |||||0 0 |0 0 ||8 |||0 0 |5 |||9 |9 |0 0 ||0 0 |||0 0 ||8 8 |8 |9 |12 12 |12 12 |12 12 |||0 0 |8 8 |||||12 12 ||10 10 |||9 ||||12 12 ||5 |9 |0 0 ||9 ||10 10 |12 12 |12 12 |||0 0 |||0 0 |0 0 |||5 |||0 0 |0 0 

||||||||||||||5 ||9 |12 12 |||0 0 |9 |0 0 |5 ||0 0 ||0 0 ||||0 0 ||5 |0 0 |9 |0 0 |||||5 |||12 12 |0 0 ||12 12 |9 |9 |||||5 ||12 12 |12 12 |9 ||||12 12 |8 |12 12 ||||9 ||5 ||9 |||0 0 |||0 0 ||9 |0 0 |0 0 |||12 12 |0 0 |||||0 0 |0 0 ||9 ||12 12 |0 0 |9 |||0 0 |||||5 |||0 0 |9 ||5 |0 0 |5 |9 |9 ||||12 12 |9 |8 |5 |5 |9 |12 12 |5 |0 0 |0 0 |||0 0 |0 0 ||12 12 |8 |12 12 |12 12 |5 ||5 |5 ||||||5 |5 |9 ||5 |5 |9 ||||||0 0 |0 0 |9 |8 |8 |1 ||9 |9 |9 |||9 |0 0 |5 |9 ||||0 0 ||12 12 |5 |5 |9 |5 |5 |12 12 |12 12 ||5 ||9 |5 ||0 0 |||0 0 |5 ||0 0 |0 0 ||0 0 ||9 9 ||5 |||0 0 |12 12 |0 0 |12 12 |0 0 |0 0 ||0 0 |||5 ||5 |||0 0 ||5 |9 ||5 |5 |0 0 ||0 0 ||1 1 |||12 12 |0 0 ||0 0 ||9 |5 ||0 0 ||9 |5 ||0 0 |12 12 ||12 12 ||||9 |9 |9 |9 ||||9 ||||0 0 |0 0 ||0 0 |||||12 12 ||12 12 ||0 0 |0 0 |0 0 ||9 |9 |9 |5 |0 0 |9 |||||||9 ||||||9 |12 12 |0 0 |12 12 ||10 10 |||||||0 0 ||0 0 |0 0 ||||8 |0 0 |0 0 |8 8 |||9 ||5 ||0 0 |0 0 |||9 ||0 0 ||0 0 |||0 0 |9 ||12 12 |||||12 12 ||

|||||||||||||||||||||||||||||||||||

...

['72', 'Self-emp-not-inc', '28865', '7th-8th', '4', 'Married-civ-spouse', 'Transport-moving', 'Husband', 'White', 'Male', '0', '0', '14', 'United-States', '<=50K']
time: 0.20403790473937988 s
acc: 75.81946545637922 %
进度: 26.328021248339972 %
['61', 'Private', '163729', 'HS-grad', '9', 'Divorced', 'Machine-op-inspct', 'Not-in-family', 'White', 'Female', '0', '0', '40', 'United-States', '<=50K']
time: 0.12751293182373047 s
acc: 75.82556087723721 %
进度: 26.334661354581673 %
['33', 'Private', '218407', 'Bachelors', '13', 'Married-civ-spouse', 'Sales', 'Husband', 'White', 'Male', '0', '0', '80', 'Columbia', '>50K']
time: 0.20005035400390625 s
acc: 75.83165322580645 %
进度: 26.341301460823374 %
['58', 'Private', '95428', 'Masters', '14', 'Divorced', 'Prof-specialty', 'Not-in-family', 'White', 'Female', '0', '0', '40', 'United-States', '<=50K']
time: 0.12853145599365234 s
acc: 75.83774250440916 %
进度: 26.347941567065075 %
['57', 'Self-emp-inc', '146103', 'HS-grad', '9', 'Married-civ-spouse', 'Sal

['37', 'Local-gov', '116736', 'HS-grad', '9', 'Married-civ-spouse', 'Craft-repair', 'Husband', 'White', 'Male', '0', '0', '40', 'United-States', '<=50K']
time: 0.18803095817565918 s
acc: 75.91204397801098 %
进度: 26.567065073041167 %
['30', 'Private', '178952', 'HS-grad', '9', 'Never-married', 'Handlers-cleaners', 'Own-child', 'White', 'Male', '0', '0', '40', 'United-States', '<=50K']
time: 0.12133121490478516 s
acc: 75.91806145390957 %
进度: 26.573705179282868 %
['34', 'Private', '156266', '11th', '7', 'Married-civ-spouse', 'Farming-fishing', 'Husband', 'Amer-Indian-Eskimo', 'Male', '0', '0', '60', 'United-States', '<=50K']
time: 0.18957233428955078 s
acc: 75.92407592407594 %
进度: 26.58034528552457 %
['22', 'Private', '163519', 'Some-college', '10', 'Never-married', 'Handlers-cleaners', 'Own-child', 'White', 'Male', '0', '0', '25', 'United-States', '<=50K']
time: 0.20303106307983398 s
acc: 75.93008739076154 %
进度: 26.58698539176627 %
['18', 'Private', '296090', 'HS-grad', '9', 'Never-marrie

['36', 'Private', '251091', 'Bachelors', '13', 'Married-civ-spouse', 'Prof-specialty', 'Husband', 'White', 'Male', '0', '0', '60', 'United-States', '>50K']
time: 0.19603991508483887 s
acc: 75.9534422981674 %
进度: 26.806108897742362 %
['71', 'Private', '149950', 'HS-grad', '9', 'Widowed', 'Priv-house-serv', 'Unmarried', 'White', 'Female', '0', '0', '20', 'United-States', '<=50K']
time: 0.21105623245239258 s
acc: 75.9593958900718 %
进度: 26.812749003984063 %
['32', 'Private', '464621', 'Some-college', '10', 'Never-married', 'Farming-fishing', 'Own-child', 'Amer-Indian-Eskimo', 'Male', '0', '0', '40', 'United-States', '<=50K']
time: 0.12651610374450684 s
acc: 75.96534653465346 %
进度: 26.819389110225764 %
['43', 'Private', '170230', 'HS-grad', '9', 'Separated', 'Adm-clerical', 'Unmarried', 'White', 'Female', '0', '0', '40', 'United-States', '<=50K']
time: 0.20553326606750488 s
acc: 75.97129423410047 %
进度: 26.826029216467465 %
['33', 'Private', '100294', 'HS-grad', '9', 'Separated', 'Craft-repa

['32', 'Private', '170983', 'HS-grad', '9', 'Never-married', 'Sales', 'Not-in-family', 'White', 'Male', '0', '0', '40', 'United-States', '<=50K']
time: 0.22377395629882812 s
acc: 76.07361963190185 %
进度: 27.051792828685258 %
['23', 'Self-emp-not-inc', '121407', 'HS-grad', '9', 'Never-married', 'Handlers-cleaners', 'Own-child', 'White', 'Female', '0', '0', '35', 'United-States', '<=50K']
time: 0.1305239200592041 s
acc: 76.07948969578018 %
进度: 27.05843293492696 %
['40', 'Local-gov', '210275', 'HS-grad', '9', 'Never-married', 'Adm-clerical', 'Other-relative', 'Black', 'Female', '0', '0', '40', 'United-States', '<=50K']
time: 0.23583555221557617 s
acc: 76.08535688005887 %
进度: 27.06507304116866 %
['21', 'Private', '116358', 'HS-grad', '9', 'Never-married', 'Other-service', 'Own-child', 'Asian-Pac-Islander', 'Male', '0', '0', '40', 'Philippines', '<=50K']
time: 0.14104747772216797 s
acc: 76.0912211868563 %
进度: 27.07171314741036 %
['48', 'Private', '189123', 'Masters', '14', 'Never-married', '

KeyboardInterrupt: 